In [41]:
from hello import hello3
from matplotlib import pyplot as plt
import numpy
from officelib.xllib import *
import datetime

In [28]:
def date(ts):
    return datetime.datetime.fromtimestamp(ts/1000).strftime("%m/%d/%y %I:%M:%S %p")

def td2xld(data):
    td = data['last']
    l = [("Time", "DOPV(%)")]
    l.extend((date(ts), pv) for ts, pv in td)
    return l

def paste(cells, xld):
    tl = cells.Range("A1")
    tr = tl.Offset(1, len(xld[0]))
    br = tr.Offset(len(xld), 1)
    cells.Range(tl, br).Value = xld
    

In [31]:
def main():
    ip = '71.189.82.196:85'
    h = hello3.HelloApp(ip)
    data = h.getTrendData('12hr', 'do')
    xld = td2xld(data)
    xl, wb, ws, cells = xlObjs()
    paste(cells, xld)
    wb.Name = "Foo.Bar"

In [43]:
xl = Excel()
for f in xl.RecentFiles:
    if f.Name.lower().startswith("80l do decay"):
        wb = f.Open()
        break
else:
    raise ValueError("uhoh")
ws = wb.Worksheets(1)
cells = ws.Cells

In [44]:
xc = cells.Range("BR1")
yc = cells.Range("BT1")
data = []
topleft = cells.Range("BV6")
with screen_lock(xl):
    for i in range(54400, 54600, 1):
        x = i/10000
        if not i % 100:
            print("\rtesting x=%.3f"%x, end="")
        xc.Value = x
        y = yc.Value
        data.append((x,y))
    br = topleft.Offset(len(data), 2)
    cells.Range(topleft, br).Value = data
    print("done")

testing x=5.450done
